# Create a Model Serving Endpoint

In [0]:
import os
import requests
import pandas as pd
import numpy as np
import json
from PIL import Image
from dotenv import load_dotenv

load_dotenv()  
DATABRICKS_TOKEN = os.getenv("DATABRICKS_TOKEN")

# Databricks API & Model Endpoint
databricks_instance = 'adb-1987745085569100.0.azuredatabricks.net'
model_name = 'VITdogClassfication'
url = f'https://{databricks_instance}/serving-endpoints/{model_name}/invocations'

headers = {
    "Authorization": f"Bearer {DATABRICKS_TOKEN}"
    # Content-Type는 requests의 json= 인자가 자동으로 넣어줍니다.
}

def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    
    # (224,224,3)
    image = image.resize((224, 224))
    image_array = np.array(image, dtype=np.float32) / 255.0  
    
    #(224,224,3) > (3,224,224) 
    image_array = image_array.transpose(2,0,1)

    # (3,224,224)  > (1, 3,224,224) 
    image_array = np.expand_dims(image_array, axis=0)

    return image_array.astype(np.float32)


def create_payload_from_ndarray(arr: np.ndarray) -> dict:
    # Databricks/MLflow pyfunc & 모델 서빙이 이해하는 표준 포맷
    return {"inputs": arr.tolist()}    

    
def score_model_ndarray(arr: np.ndarray):
    payload = create_payload_from_ndarray(arr)
    resp = requests.post(url, headers=headers, json=payload)  # use json=, not data=
    if resp.status_code != 200:
        raise Exception(f"Request failed: {resp.status_code}, {resp.text}")
    return resp.json()

# Load an image and make predictions
mount_path = "/mnt/vision-test/"
image_path = f"{mount_path}images"
files = [file.path.replace("dbfs:/","/dbfs/") for file in dbutils.fs.ls(image_path)]
image_path_sample = files.pop()


input_data = preprocess_image(image_path_sample)
print(f"input data: {input_data.shape}")



input data: (1, 3, 224, 224)


In [0]:
response = score_model_ndarray(input_data)
response

{'predictions': [[-2.7430965900421143, 0.44489455223083496]]}

In [0]:
type(response['predictions'])

list

In [0]:
# Class 0 -> abnormal
# Class 1 -> normal

# Softmax
def softmax(logits):
    exp_logits = np.exp(logits - np.max(logits))
    return exp_logits / np.sum(exp_logits)


def post_process(response):
    logits = response['predictions'][0]
    return softmax(np.array(logits))

probabilities = post_process(response)
print('Class logits: ', response['predictions'][0])
print('Class probabilities: ', probabilities)

Class logits:  [-2.7430965900421143, 0.44489455223083496]
Class probabilities:  [0.03962015 0.96037985]
